In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,44279
2,Huelva,Confirmados PDIA 14 días,1511
3,Huelva,Tasa PDIA 14 días,"294,44433618489"
4,Huelva,Confirmados PDIA 7 días,514
5,Huelva,Tasa PDIA 7 dias,"100,16173977434379"
6,Huelva,Total Confirmados,44477
7,Huelva,Curados,40150
8,Huelva,Fallecidos,403


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  44279.0


/tmp/ipykernel_33641/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12298.0


/tmp/ipykernel_33641/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_33641/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_33641/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_33641/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1262 personas en los últimos 7 días 

Un positivo PCR cada 395 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,44279.0,514.0,1511.0,513170.0,100.161740,294.444336,133.0
Huelva-Costa,26159.0,326.0,976.0,289548.0,112.589277,337.077099,95.0
Condado-Campiña,13792.0,131.0,385.0,156231.0,83.850196,246.429966,29.0
Huelva (capital),12298.0,114.0,364.0,143837.0,79.256380,253.064232,29.0
Cartaya,1848.0,50.0,120.0,20083.0,248.966788,597.520291,12.0
Ayamonte,1983.0,28.0,96.0,21104.0,132.676270,454.890068,11.0
Isla Cristina,3026.0,32.0,125.0,21393.0,149.581639,584.303277,10.0
Sierra de Huelva-Andévalo Central,3917.0,53.0,130.0,67391.0,78.645516,192.904097,9.0
Lepe,2940.0,32.0,122.0,27880.0,114.777618,437.589670,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Linares de la Sierra,11.0,1.0,6.0,262.0,381.679389,2290.076336,0.0
Cañaveral de León,29.0,3.0,8.0,396.0,757.575758,2020.202020,1.0
Almendro (El),57.0,8.0,14.0,840.0,952.380952,1666.666667,0.0
Rosal de la Frontera,71.0,11.0,24.0,1697.0,648.202711,1414.260460,2.0
Cabezas Rubias,52.0,4.0,8.0,706.0,566.572238,1133.144476,1.0
Escacena del Campo,231.0,8.0,23.0,2287.0,349.803236,1005.684303,3.0
Sanlúcar de Guadiana,34.0,3.0,3.0,403.0,744.416873,744.416873,NaN
San Silvestre de Guzmán,21.0,4.0,4.0,614.0,651.465798,651.465798,NaN
Cartaya,1848.0,50.0,120.0,20083.0,248.966788,597.520291,12.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Bollullos Par del Condado,1257.0,3.0,22.0,14387.0,20.852158,152.915827,1.0,0.136364
Niebla,356.0,3.0,18.0,4116.0,72.886297,437.317784,0.0,0.166667
Linares de la Sierra,11.0,1.0,6.0,262.0,381.679389,2290.076336,0.0,0.166667
San Juan del Puerto,750.0,8.0,37.0,9411.0,85.006907,393.156944,1.0,0.216216
Rociana del Condado,656.0,2.0,9.0,7939.0,25.192090,113.364404,1.0,0.222222
Palos de la Frontera,987.0,3.0,13.0,11742.0,25.549310,110.713677,0.0,0.230769
Almonte,2225.0,15.0,59.0,24507.0,61.207002,240.747542,3.0,0.254237
Isla Cristina,3026.0,32.0,125.0,21393.0,149.581639,584.303277,10.0,0.256000
Lepe,2940.0,32.0,122.0,27880.0,114.777618,437.589670,8.0,0.262295
